In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta

In [2]:
chromedriver = "C:\Program Files\Google\Chrome\Application/chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
url_list = []
date_list = []
list_of_loc =  ["FRA-RUH","VIE-RUH","ORY-RUH",
               "GVA-RUH","AMS-RUH","BUD-RUH",
               "BRU-RUH","LHR-RUH","MAD-RUH",
               "PRG-RUH","LIS-RUH",]
def make_flight_url(starting_date):

    base_url1 = 'https://www.almosafer.com/en/flights/'
    base_url2 = '/Economy/1Adult/'
    for k in list_of_loc:
        for i in range(0,7):
            starting_date_2 = (starting_date + timedelta(days=i))
            date_str = str(starting_date_2.year) + "-" + str(starting_date_2.month).zfill(2) + "-" + str(starting_date_2.day).zfill(2)
            date_list.append(date_str)
            search_url = (base_url1 + k + "/" + date_str + base_url2)
            url_list.append(search_url)
    return url_list , date_list

starting_date = '12/01/21'
starting_date = datetime.strptime(starting_date, '%m/%d/%y')
make_flight_url(starting_date)

(['https://www.almosafer.com/en/flights/FRA-RUH/2021-12-01/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-02/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-03/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-04/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-05/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-06/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/FRA-RUH/2021-12-07/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-01/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-02/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-03/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-04/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-05/Economy/1Adult/',
  'https://www.almosafer.com/en/flights/VIE-RUH/2021-12-06/Economy/1Adult/',

In [6]:
columns = ["Flight_Number","Ariline","Class","Date","Day","Time_of_Landing","Time_of_arrival","Number_of_Stops","Duration","Baggage","Location","Destination","Price"]
flight_df = pd.DataFrame(columns=columns)

i = 0

for url in url_list[:5]:
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(2)
    
    price_list =[]
    airline_list = []
    flight_num_list = []
    time_landing_list = []
    time_arrival_list = []
    Duration_list = []
    Baggage_list =[]
    stops_list =[]
    

    for j in range(8):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source)
    soup

    for rest in soup.find_all('div', class_='sc-cSYcjD hXnhxm')[:]: 
        price_list.append(rest.text)

    for rest in soup.find_all('div', class_='sc-dEfkYy dunONY')[:]: 
        airline_list.append(rest.find('div').text)

    for rest in soup.find_all('div', class_='sc-dEfkYy dunONY')[:]: 
        flight_num_list.append(rest.find_all('div')[-1].text)

    for rest in soup.find_all('div', class_='sc-bIqbHp aPzHZ sc-kpOJdX kAzjOl col-2')[:]: 
        time_landing_list.append(rest.find("div").text)

    for rest in soup.find_all('div', class_='sc-bIqbHp gwjyCQ sc-kpOJdX kAzjOl col-2')[:]: 
        time_arrival_list.append(rest.find("div").text)

    for rest in soup.find_all('div',class_="sc-jxGEyO dfxDXk sc-kpOJdX kAzjOl col-5")[:]:
        stops_list.append(rest.find("span").text)
        
    for rest in soup.find_all('span', class_='sc-FQuPU kdpQLw')[:]: 
        Duration_list.append(rest.text)

    for rest in soup.find_all('span', class_='sc-iuDHTM cnrMzL')[:]: 
        Baggage_list.append(rest.text)

    for M in range(len(flight_num_list)):
        flight_df.loc[len(flight_df)] = {"Flight_Number":flight_num_list[M],
                             "Ariline":airline_list[M],
                             "Class":"Economy",
                             "Date":date_list[i],
                             "Day":soup.find('th', class_='sc-cBrjTV gRVZVE').find("div").text,
                             "Time_of_Landing":time_landing_list[M],
                             "Time_of_arrival":time_arrival_list[M],
                             "Number_of_Stops":stops_list[M],   
                             "Duration":Duration_list[M],
                             "Baggage":Baggage_list[M],
                             "Location":soup.find('div', class_='sc-yZwTr bmGDAz').text,
                             "Destination":"RUH",
                             "Price":price_list[M]}
    
    i += 1
    
    driver.close()
flight_df.to_csv("Economy_FLIGHTS")

<ipython-input-6-302ef148e6d0>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [9]:
flight_df.Number_of_Stops.value_counts()

1 Stop      174
2 Stops      54
Direct        7
+2 Stops      1
Name: Number_of_Stops, dtype: int64